In [1]:
from sqlalchemy.engine.url import URL

DB_USERNAME = "bgd22_7666537" # YOUR USER NAME
DB_PASSWORD = "dKsqS87o" # YOUR PASSWORD
DB_SCHEMA = "public"
DB_HOSTNAME="gip.itc.utwente.nl"
DB_PORT=5434
DB_DATABASE="c211"



postgres_db = {'drivername': 'postgresql',
               'username': DB_USERNAME,
               'password': DB_PASSWORD,
               'host': DB_HOSTNAME,
               'port': DB_PORT,
               'database': DB_DATABASE}

db_url = URL.create(**postgres_db)

In [2]:
from sqlalchemy import create_engine

engine = create_engine(
    db_url, isolation_level="AUTOCOMMIT", pool_size=10, max_overflow=20)

In [3]:
import pandas as pd
dataout_path = '0_data/' # Should create this folder to store data.

In [4]:
dfspatial_query = "SELECT b.*,  bio.aves2012_2016, pop.pop001, pop.pop009, pop.pop025, pop.pop049, pop.pop081, pop.pop121, pop.pop169, pop.pop225 "\
    "FROM public.block AS b, public.biodiversity AS bio, public.block_demography AS pop "\
    "WHERE b.latit != '0' "\
    "AND b.block = bio.block AND b.block = pop.block;"
# Don't use landuse here, because block is not unique in block_landuse.
dfspatial = pd.read_sql(dfspatial_query, engine)  

# Connect dfspatial and road data.
dfroad_length = pd.read_csv(dataout_path + 'roadnowater.csv')

dfspatialall = pd.merge(dfspatial, dfroad_length, on = 'block')

In [5]:
df2017stfeatures_query = "SELECT dy.odate AS obsdate, dy.doy, dy.dow, dy.natholiday, prec.block, prec.precip, temp.temper "\
    "FROM public.days AS dy, public.precipitation AS prec, public.temperature as temp "\
    "WHERE dy.mdate = prec.dtime AND dy.mdate = temp.dtime AND prec.dtime = temp.dtime AND prec.block = temp.block "\
    "AND dy.odate >= '2017-01-01' AND dy.odate <= '2017-06-30';"

df2017stfeatures = pd.read_sql(df2017stfeatures_query, engine)  

In [6]:
df2017pred_features = pd.merge(df2017stfeatures, dfspatialall, on = 'block')
df2017pred_features.head()

,obsdate,doy,dow,natholiday,block,precip,temper,longit,latit,geom,...,aves2012_2016,pop001,pop009,pop025,pop049,pop081,pop121,pop169,pop225,roads_nowater_length
0,2017-01-01,1,0,True,62,5.560510,1.37155,15,369,010300002040710000010000000500000000000000004C...,...,13,30,1810,2070,2665,4025,5060,5330,7735,1844.323239
1,2017-01-02,2,1,False,62,0.375260,7.20379,15,369,010300002040710000010000000500000000000000004C...,...,13,30,1810,2070,2665,4025,5060,5330,7735,1844.323239
2,2017-01-03,3,2,False,62,0.632896,6.22569,15,369,010300002040710000010000000500000000000000004C...,...,13,30,1810,2070,2665,4025,5060,5330,7735,1844.323239
3,2017-01-04,4,3,False,62,0.777120,7.72550,15,369,010300002040710000010000000500000000000000004C...,...,13,30,1810,2070,2665,4025,5060,5330,7735,1844.323239
4,2017-01-05,5,4,False,62,0.033960,5.42826,15,369,010300002040710000010000000500000000000000004C...,...,13,30,1810,2070,2665,4025,5060,5330,7735,1844.323239


In [7]:
df2017pred_features.shape

(5929365, 21)

In [8]:
df2017pred_features.to_csv(dataout_path+'2017pred_stfeatures.csv')